# Sorting Algorithms

In [41]:
from fiddle import *
configure_for_jupyter()
sort = build_one("sorting/sort.cpp")

In [42]:
display(sort.source("bubble_sort"))

uint64_t* bubble_sort(uint64_t size)
{
	uint64_t * list = random_array(size);
	start_measurement();
	uint64_t temp;
	for(uint64_t i=0; i<size; i++)
	{
		for(uint64_t j=size-1; j>i; j--)
		{
			if(list[j]<list[j-1])
			{
				temp=list[j-1];
				list[j-1]=list[j];
				list[j]=temp;
			}
		}
	}
	end_measurement();
	CHECK(list);
	return NULL;
}

In [43]:
sort.source("insertion_sort")

uint64_t* insertion_sort(uint64_t size)
{
	uint64_t * list = random_array(size);
	start_measurement();
	for(uint64_t j=1;j<size;j++)
	{
		uint64_t key=list[j];
		uint64_t i = j-1;
		while(list[i] > key)
		{
			list[i+1]=list[i];
			if (i == 0)
				break;
			i=i-1;
		}
		list[i+1]=key;

	}
	end_measurement();
	CHECK(list);
	return NULL;
}

In [44]:
sort.source("random_array")

uint64_t*random_array(uint64_t size) {
	uint64_t * data = new uint64_t[size];
	auto rng = fast_URBG();
	for(unsigned int i = 0; i < size; i++) {
		std::shuffle(data, &data[size], rng);
	}
	return data;
}

In [45]:
r = run(product([sort], ["bubble_sort", "insertion_sort"], map_product(size=range(400, 4000, 400)))).as_df()
r

,function,size,ET
0,bubble_sort,400,0.000340
1,bubble_sort,800,0.000896
2,bubble_sort,1200,0.002014
3,bubble_sort,1600,0.003580
4,bubble_sort,2000,0.005086
5,bubble_sort,2400,0.007473
6,bubble_sort,2800,0.010909
7,bubble_sort,3200,0.009796
8,bubble_sort,3600,0.012242
9,insertion_sort,400,0.000002
